In [40]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import folium
import requests
from folium.plugins import HeatMap, HeatMapWithTime

In [41]:
df = pd.read_csv("Dataset/Sample_NaStreet_Removed.csv")
df_street=pd.read_csv("street.csv")

In [42]:
#df.iloc[:,25:30].head()
#df.isnull().sum()
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Summons Number', 'Plate ID',
       'Registration State', 'Plate Type', 'Issue Date', 'Violation Code',
       'Vehicle Body Type', 'Vehicle Make', 'Issuing Agency', 'Street Code1',
       'Street Code2', 'Street Code3', 'Vehicle Expiration Date',
       'Violation Location', 'Violation Precinct', 'Issuer Precinct',
       'Issuer Code', 'Issuer Command', 'Issuer Squad', 'Violation Time',
       'Time First Observed', 'Violation County',
       'Violation In Front Of Or Opposite', 'House Number', 'Street Name',
       'Intersecting Street', 'Date First Observed', 'Law Section',
       'Sub Division', 'Violation Legal Code', 'Days Parking In Effect    ',
       'From Hours In Effect', 'To Hours In Effect', 'Vehicle Color',
       'Unregistered Vehicle?', 'Vehicle Year', 'Meter Number',
       'Feet From Curb', 'Violation Post Code', 'Violation Description',
       'No Standing or Stopping Violation', 'Hydrant Violation',
       'Double Parking Viola

In [43]:
print(len(df))
invalid = df.loc[(df['Latitude']=='n') & (df['Longitude']=='o')]
print(len(invalid))
df = df[(df['Latitude']!='n') & (df['Longitude']!='o')]
df = df[pd.notnull(df.Hour)]
print(len(df))

1079918
2665
1077238


In [44]:
df_hour=[]
for hour in df.Hour.sort_values().unique():
    df_hour.append(df.loc[df.Hour == hour, ['Latitude', 'Longitude', 'count']].groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist())

In [45]:
df.Hour.sort_values().unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23.])

In [46]:
def generateBaseMap(default_location=[40.693943, -73.985880], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [47]:
base_map = generateBaseMap(default_zoom_start=11)
HeatMapWithTime(df_hour, radius=15, min_opacity=0, max_opacity=0.6, use_local_extrema=False).add_to(base_map)

In [48]:
base_map.save("hourly map.html")

In [49]:
df['Issue Date']=pd.to_datetime(df['Issue Date'])
df['Week']=[i.week for i in df['Issue Date']]

In [50]:
df_week=[]
for week in df.Week.sort_values().unique():
    df_week.append(df.loc[df.Week == week, ['Latitude', 'Longitude', 'count']].groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist())

In [51]:
df.Week.sort_values().unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53], dtype=int64)

In [52]:
base_map = generateBaseMap(default_zoom_start=11)
HeatMapWithTime(df_week, radius=15, min_opacity=0, max_opacity=0.6, use_local_extrema=False).add_to(base_map)

In [53]:
base_map.save("weekly map.html")

In [54]:
df['Month']=[i.month for i in df['Issue Date']]

In [55]:
df_month=[]
for month in df.Month.sort_values().unique():
    df_month.append(df.loc[df.Month == month, ['Latitude', 'Longitude', 'count']].groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist())

In [56]:
base_map = generateBaseMap(default_zoom_start=11)
HeatMapWithTime(df_month, radius=15, min_opacity=0, max_opacity=0.6, use_local_extrema=False).add_to(base_map)

In [57]:
base_map.save("monthly map.html")

In [58]:
df['Day']=[i.day for i in df['Issue Date']]

In [65]:
df_day=[]
for day in df.Day[df.Month==6].sort_values().unique():
    df_day.append(df.loc[df.Day == day, ['Latitude', 'Longitude', 'count']].groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist())

In [66]:
base_map = generateBaseMap(default_zoom_start=11)
HeatMapWithTime(df_day, radius=15, min_opacity=0, max_opacity=0.6, use_local_extrema=False).add_to(base_map)

In [64]:
base_map.save("daily map.html")